In [1]:
import numpy as np
import pandas as pd
import os
import gc

from PIL import Image
import cv2

import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

gc.enable()

rand_seed = 1120

In [2]:
dataset_folder = "/workspace/Kaggle/KolektorSDD"

In [3]:
dataset_inputs = {}
dataset_labels = {}
defective_count = 0
for folder in os.listdir(dataset_folder):
    for filename in os.listdir(f"{dataset_folder}/{folder}"):
        # print(folder, filename)

        if ".bmp" in filename:
            part_id = filename.split("_")[0]

            label_img = Image.open(f"{dataset_folder}/{folder}/{filename}")
            label_img = np.array(label_img)
            # print(label_img.shape)

            if np.sum(label_img) > 0:
                defective_count += 1

                print(f"{folder}-{part_id} has defect:", np.sum(label_img) > 0)

                dataset_labels[f"{folder}-{part_id}"] = label_img

        else:
            part_id = filename.split(".")[0]

            input_img = Image.open(
                f"{dataset_folder}/{folder}/{filename}").convert('L')
            input_img = np.array(input_img)
            # print(input_img.shape)

            dataset_inputs[f"{folder}-{part_id}"] = input_img

kos28-Part3 has defect: True
kos41-Part7 has defect: True
kos18-Part3 has defect: True
kos01-Part5 has defect: True
kos20-Part4 has defect: True
kos49-Part2 has defect: True
kos05-Part5 has defect: True
kos24-Part1 has defect: True
kos42-Part3 has defect: True
kos09-Part6 has defect: True
kos13-Part3 has defect: True
kos48-Part5 has defect: True
kos30-Part0 has defect: True
kos31-Part1 has defect: True
kos33-Part6 has defect: True
kos46-Part7 has defect: True
kos35-Part5 has defect: True
kos25-Part4 has defect: True
kos37-Part0 has defect: True
kos10-Part3 has defect: True
kos11-Part4 has defect: True
kos43-Part1 has defect: True
kos21-Part5 has defect: True
kos45-Part3 has defect: True
kos16-Part5 has defect: True
kos19-Part5 has defect: True
kos03-Part2 has defect: True
kos17-Part5 has defect: True
kos40-Part5 has defect: True
kos22-Part6 has defect: True
kos15-Part3 has defect: True
kos50-Part4 has defect: True
kos38-Part1 has defect: True
kos38-Part0 has defect: True
kos26-Part5 ha

In [4]:
defective_count

52

In [5]:
# https://stackoverflow.com/questions/4808221/is-there-a-bounding-box-function-slice-with-non-zero-values-for-a-ndarray-in
def crop_bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    return img[ymin:ymax + 1, xmin:xmax + 1], (ymin, ymax, xmin, xmax)


resized_dataset = {}
for key, input_image in dataset_inputs.items():

    if key in dataset_labels:
        label_image = dataset_labels[key]

        # Crop center 500px
        bbox_image, (ymin, ymax, xmin, xmax) = crop_bbox(label_image)

        y_center = round((ymax + ymin) / 2)
        x_center = round((xmax + xmin) / 2)

        # print(input_image.shape, label_image.shape)

        # print(ymin, ymax, xmin, xmax)

        input_image = input_image[y_center - 250:y_center + 250, :]
        label_image = label_image[y_center - 250:y_center + 250, :]

        # print(input_image.shape, label_image.shape)

        assert input_image.shape == label_image.shape == (500, 500)

        input_image = cv2.resize(input_image, (128, 128))
        label_image = cv2.resize(label_image, (128, 128))

        assert input_image.shape == label_image.shape == (128, 128)

        resized_dataset[key] = (input_image, label_image)

        # plt.imshow(input_image, cmap='gray')
        # plt.show()

        # plt.imshow(label_image, cmap='gray')
        # plt.show()

        # plt.imshow(bbox_image, cmap='gray')
        # plt.show()
    else:
        y_center = round(input_image.shape[0] / 2)

        input_image = input_image[y_center - 250:y_center + 250, :]

        assert input_image.shape == (500, 500)

        input_image = cv2.resize(input_image, (128, 128))

        assert input_image.shape == (128, 128)

        resized_dataset[key] = (input_image, None)

In [6]:
len(resized_dataset)

399

In [19]:
# https://stackoverflow.com/questions/4808221/is-there-a-bounding-box-function-slice-with-non-zero-values-for-a-ndarray-in
def crop_bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    return img[ymin:ymax + 1, xmin:xmax + 1], (ymin, ymax, xmin, xmax)


data_x = []
data_y = []
for key, input_image in dataset_inputs.items():

    if key in dataset_labels:
        label_image = dataset_labels[key]

        # Crop center 500px
        bbox_image, (ymin, ymax, xmin, xmax) = crop_bbox(label_image)

        y_center = round((ymax + ymin) / 2)
        x_center = round((xmax + xmin) / 2)

        input_image = input_image[y_center - 250:y_center + 250, :]

        input_image = cv2.resize(input_image, (128, 128)).astype(np.float32)

        assert input_image.shape == (128, 128)

        data_x.append(input_image[np.newaxis, :, :, np.newaxis] / 255)
        data_y.append(1)
    else:
        y_center = round(input_image.shape[0] / 2)

        input_image = input_image[y_center - 250:y_center + 250, :]

        input_image = cv2.resize(input_image, (128, 128)).astype(np.float32)

        assert input_image.shape == (128, 128)

        data_x.append(input_image[np.newaxis, :, :, np.newaxis] / 255)
        data_y.append(0)

data_x = np.concatenate(data_x, axis=0)
data_y = np.array(data_y, dtype=np.int32)

In [20]:
data_x.shape, data_y.shape

((399, 128, 128, 1), (399,))

In [21]:
data_x.min(), data_x.max()

(0.02745098, 1.0)